### Necessary libraries

In [17]:
import pandas as pd
import geopy.distance
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

### Listings - Dataset

In [129]:
listings = pd.read_csv("data/listings.csv")

listings_cleaned = listings[["id", "name", "neighbourhood_cleansed", "latitude", "longitude", "property_type",
                             "room_type", "accommodates", "bedrooms", "price","number_of_reviews",
                             "review_scores_rating", "review_scores_location", "reviews_per_month"]]
listings_cleaned.head()




,id,name,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month
0,15883,b&b near Old Danube river,Donaustadt,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,1.0,$110.00,14,4.71,4.71,0.15
1,38768,central cityapartement- wifi- nice neighbourhood,Leopoldstadt,48.21924,16.37831,Entire rental unit,Entire home/apt,5,1.0,$69.00,350,4.75,4.75,2.50
2,40625,"Near Palace Schönbrunn, Apt. 1",Rudolfsheim-Fnfhaus,48.18434,16.32701,Entire rental unit,Entire home/apt,6,2.0,$145.00,181,4.83,4.59,1.23
3,392757,VCA3 Palais Brambilla - studio with city views,Innere Stadt,48.21496,16.37161,Entire rental unit,Entire home/apt,2,1.0,$100.00,100,4.64,4.89,0.79
4,51287,little studio- next to citycenter- wifi- nice ...,Leopoldstadt,48.21778,16.37847,Entire rental unit,Entire home/apt,3,NaN,$68.00,347,4.65,4.86,2.45


In [130]:
listings_cleaned.loc[:,'neighbourhood_cleansed'] = listings_cleaned.loc[:,'neighbourhood_cleansed'].apply(
    lambda x: "Rudolfsheim-Fünfhaus" if str(x).__contains__("Rudolfsheim") else x
)
listings_cleaned.head()
mapping = pd.read_csv("data/district_to_post.csv")
merged = listings_cleaned.merge(mapping, left_on="neighbourhood_cleansed", right_on="name", how="inner")
merged.head()
merged.drop(["neighbourhood_cleansed", "name_y"], inplace=True, axis=1)
merged.rename({"name_x":"name"}, inplace=True, axis=1)
listings_cleaned = merged

C:\Users\Locke\AppData\Local\Temp\ipykernel_17096\1619392840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_cleaned.loc[:,'neighbourhood_cleansed'] = listings_cleaned.loc[:,'neighbourhood_cleansed'].apply(


,id,name,latitude,longitude,property_type,room_type,accommodates,bedrooms,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month,post
0,15883,b&b near Old Danube river,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,1.0,$110.00,14,4.71,4.71,0.15,1220
1,813826,apartment2rent panaroma - UNO City,48.23796,16.42418,Entire rental unit,Entire home/apt,4,NaN,$82.00,10,4.80,5.00,0.10,1220
2,846009,"Nice, clean and perfect location Ap",48.23549,16.41152,Entire condo,Entire home/apt,3,1.0,$60.00,72,4.77,4.86,0.61,1220
3,1384870,Nice bedroom in sweet apartment!,48.22707,16.44955,Private room in rental unit,Private room,2,1.0,$34.00,92,4.60,4.59,0.83,1220
4,3217556,Apartment Vienna B,48.21659,16.51765,Entire rental unit,Entire home/apt,4,1.0,$76.00,17,4.76,4.65,0.51,1220


In [131]:
listings_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10152 entries, 0 to 10151
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      10152 non-null  int64  
 1   name                    10144 non-null  object 
 2   latitude                10152 non-null  float64
 3   longitude               10152 non-null  float64
 4   property_type           10152 non-null  object 
 5   room_type               10152 non-null  object 
 6   accommodates            10152 non-null  int64  
 7   bedrooms                9137 non-null   float64
 8   price                   10152 non-null  object 
 9   number_of_reviews       10152 non-null  int64  
 10  review_scores_rating    8503 non-null   float64
 11  review_scores_location  8421 non-null   float64
 12  reviews_per_month       8503 non-null   float64
 13  post                    10152 non-null  int64  
dtypes: float64(6), int64(4), object(4)
mem

Price should be numeric, therefore we will convert it to numeric form. First, lets have a look into it:


In [132]:
listings_cleaned[['price']].head(5)

,price
0,$110.00
1,$82.00
2,$60.00
3,$34.00
4,$76.00


Before converting to numeric, we will remove the currency and then do the conversion.

In [133]:
listings_cleaned['price'] = listings_cleaned['price'].str.extract('(\d+)', expand=False)
listings_cleaned[['price']] = listings_cleaned[['price']].apply(pd.to_numeric)

#### Handling missing values

As seen below there are some missing values in our dataset:

In [134]:
listings_cleaned.isna().sum()

id                           0
name                         8
latitude                     0
longitude                    0
property_type                0
room_type                    0
accommodates                 0
bedrooms                  1015
price                        0
number_of_reviews            0
review_scores_rating      1649
review_scores_location    1731
reviews_per_month         1649
post                         0
dtype: int64

In [135]:
# looking into the missing values
listings_cleaned[listings_cleaned.isna().any(axis = 1)]

,id,name,latitude,longitude,property_type,room_type,accommodates,bedrooms,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month,post
1,813826,apartment2rent panaroma - UNO City,48.237960,16.424180,Entire rental unit,Entire home/apt,4,NaN,82,10,4.80,5.00,0.10,1220
9,4058386,Apartment Vienna Loft,48.217140,16.518090,Entire loft,Entire home/apt,4,NaN,85,86,4.66,4.58,1.00,1220
11,4781153,Room in a quiet house near the park,48.202870,16.519740,Private room in home,Private room,2,1.0,48,0,NaN,NaN,NaN,1220
18,5623462,Ruhige Maisonette in U-Bahnnähe,48.223850,16.489930,Entire rental unit,Entire home/apt,2,1.0,60,0,NaN,NaN,NaN,1220
20,6082758,NICE APARTMENT + air conditioning,48.227810,16.455850,Entire rental unit,Entire home/apt,2,NaN,79,133,4.94,4.64,1.49,1220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10115,684612125199203188,DAREBELL Apartment Calvigasse: Top 8,48.140880,16.313433,Entire rental unit,Entire home/apt,2,NaN,105,3,4.67,4.33,3.00,1230
10118,694670571352670381,Sonnige Dachgeschosswohnung mit großer Terrasse,48.143623,16.308706,Entire rental unit,Entire home/apt,2,1.0,60,0,NaN,NaN,NaN,1230
10119,695565932820425184,Atmospheric Studio with Kitchen in Vienna 4,48.149028,16.337260,Entire condo,Entire home/apt,2,1.0,85,0,NaN,NaN,NaN,1230
10131,683168261456239114,Grünruhelage nahe Wien Innenstadt,48.131196,16.279044,Private room in rental unit,Private room,2,1.0,50,0,NaN,NaN,NaN,1230


In [136]:
# trying sth out

listings_cleaned[(listings_cleaned['number_of_reviews']==0) & (listings_cleaned['review_scores_rating'].isna())]

,id,name,latitude,longitude,property_type,room_type,accommodates,bedrooms,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month,post
11,4781153,Room in a quiet house near the park,48.202870,16.519740,Private room in home,Private room,2,1.0,48,0,NaN,NaN,NaN,1220
18,5623462,Ruhige Maisonette in U-Bahnnähe,48.223850,16.489930,Entire rental unit,Entire home/apt,2,1.0,60,0,NaN,NaN,NaN,1220
33,8138284,Student Apartment 2 separate Rooms,48.227940,16.457430,Entire place,Entire home/apt,2,2.0,36,0,NaN,NaN,NaN,1220
40,9900495,Close to underground line u2,48.224630,16.452860,Private room in rental unit,Private room,1,1.0,40,0,NaN,NaN,NaN,1220
42,10135734,Маленькая однокомнатная квартира,48.212830,16.441540,Entire rental unit,Entire home/apt,1,1.0,30,0,NaN,NaN,NaN,1220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10111,640967788256740265,DAREBELL Apartment Calvigasse: Top 18,48.141320,16.311500,Entire rental unit,Entire home/apt,2,1.0,105,0,NaN,NaN,NaN,1230
10118,694670571352670381,Sonnige Dachgeschosswohnung mit großer Terrasse,48.143623,16.308706,Entire rental unit,Entire home/apt,2,1.0,60,0,NaN,NaN,NaN,1230
10119,695565932820425184,Atmospheric Studio with Kitchen in Vienna 4,48.149028,16.337260,Entire condo,Entire home/apt,2,1.0,85,0,NaN,NaN,NaN,1230
10131,683168261456239114,Grünruhelage nahe Wien Innenstadt,48.131196,16.279044,Private room in rental unit,Private room,2,1.0,50,0,NaN,NaN,NaN,1230


In [137]:
withreviews = listings_cleaned[listings_cleaned['number_of_reviews']!=0]
withreviews[withreviews['review_scores_location'].isna()].head(5)

,id,name,latitude,longitude,property_type,room_type,accommodates,bedrooms,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month,post
59,16541320,cosy viennese apartment with garden,48.23731,16.46327,Entire rental unit,Entire home/apt,3,1.0,30,1,0.0,NaN,0.01,1220
75,19264213,Cosy Wohnung in der Nähe von Doauinsel und UN,48.24870,16.43463,Entire rental unit,Entire home/apt,1,NaN,45,1,0.0,NaN,0.44,1220
114,22368540,Studio 8b for 2 People - free parking included,48.23993,16.54627,Entire condo,Entire home/apt,1,NaN,48,1,0.0,NaN,0.02,1220
176,34384502,Smart Sleep,48.21674,16.48274,Entire rental unit,Entire home/apt,2,1.0,1,5,0.0,NaN,0.12,1220
444,5102698,Wohnung im Herzen der Stadt!,48.22220,16.38228,Entire rental unit,Entire home/apt,4,2.0,99,1,0.0,NaN,0.01,1020


### First glimpse into NAN values

1. All the missing values of column `review_scores_rating`, `reviews_per_month` are due to the `number_of_reviews` being 0. So I believe these values are MAR (Missing at Random).
2. Moreover, also 1928 out of 2024 missing value rows of `review_scores_location` are connected to the fact that the `number_of_reviews` is 0. For remaining 96,they are missing even though there are few reviews for the listing, maybe due to the fact that the guest did not rate location explictly.
3. For column `bedrooms`, I think there is no connection between this variable missing and other columns- MCAR.

#### WIENTOURISM Data Set

In [140]:
tourism = pd.read_csv("data/WIENTOURISMUSOGD.csv")
toursim_cleaned = tourism[['NAME','POSTALCODE', 'CATEGORY_NAME','SUBCATEGORY_NAME', 'GEOLAT', 'GEOLONG']]
remove_cats = ['Hotel', 'Pension', 'Appartement', 'Saisonhotel']
toursim_cleaned = toursim_cleaned[~toursim_cleaned['SUBCATEGORY_NAME'].isin(remove_cats)]
toursim_cleaned.head()

,NAME,POSTALCODE,CATEGORY_NAME,SUBCATEGORY_NAME,GEOLAT,GEOLONG
0,Porzellan,1090,"Essen, Trinken & Nightlife",Restaurant,48.219222,16.364447
1,Postamt 1150 Wien,1150,Infrastruktur,Sonstiges,48.195808,16.338125
2,Prater Alm,1020,"Essen, Trinken & Nightlife",Gasthaus & Beisl,48.215450,16.402072
3,Pratermuseum,1020,Sightseeing,Museum,48.216236,16.394952
4,Praterturm,1020,Sightseeing,Sehenswürdigkeit,48.216727,16.398029


### Joining Datasets

In [141]:
joined_df = toursim_cleaned.join(listings_cleaned, how="cross", lsuffix="_tourism", rsuffix="_listing")
joined_df.head(5)

,NAME,POSTALCODE,CATEGORY_NAME,SUBCATEGORY_NAME,GEOLAT,GEOLONG,id,name,latitude,longitude,property_type,room_type,accommodates,bedrooms,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month,post
0,Porzellan,1090,"Essen, Trinken & Nightlife",Restaurant,48.219222,16.364447,15883,b&b near Old Danube river,48.24262,16.42767,Room in bed and breakfast,Hotel room,3,1.0,110,14,4.71,4.71,0.15,1220
1,Porzellan,1090,"Essen, Trinken & Nightlife",Restaurant,48.219222,16.364447,813826,apartment2rent panaroma - UNO City,48.23796,16.42418,Entire rental unit,Entire home/apt,4,NaN,82,10,4.80,5.00,0.10,1220
2,Porzellan,1090,"Essen, Trinken & Nightlife",Restaurant,48.219222,16.364447,846009,"Nice, clean and perfect location Ap",48.23549,16.41152,Entire condo,Entire home/apt,3,1.0,60,72,4.77,4.86,0.61,1220
3,Porzellan,1090,"Essen, Trinken & Nightlife",Restaurant,48.219222,16.364447,1384870,Nice bedroom in sweet apartment!,48.22707,16.44955,Private room in rental unit,Private room,2,1.0,34,92,4.60,4.59,0.83,1220
4,Porzellan,1090,"Essen, Trinken & Nightlife",Restaurant,48.219222,16.364447,3217556,Apartment Vienna B,48.21659,16.51765,Entire rental unit,Entire home/apt,4,1.0,76,17,4.76,4.65,0.51,1220


#### Computing distance  between each listing and each POI as euclidean distance

In [33]:
joined_df['distance'] = joined_df.apply(lambda x: geopy.distance.geodesic((x['GEOLONG'], x['GEOLAT']),
                                (x['longitude'], x['latitude'])).km, axis=1)
joined_df.head()
joined_df.to_csv("data/joined_df.csv")

KeyboardInterrupt: 

In [ ]:
borders = pd.read_csv("data/borders.csv").to_numpy()
def check_distance(x):
    check_tuple = np.array([x['post'], x['POSTALCODE']])
    borders_dist = (borders==check_tuple).all(axis=1).any()
    if int(x["post"]) == int(x["POSTALCODE"]) or borders_dist:
       return True
    else:
        return False

def calculate_distance(x):
     x["distance"] = geopy.distance.geodesic((x['GEOLONG'], x['GEOLAT']),
                                (x['longitude'], x['latitude'])).km
     return x

mask = joined_df.apply(lambda x:check_distance(x), axis=1)
df = joined_df.loc[mask,:]
df = df.apply(lambda x: calculate_distance(x), axis=1)
df.head()
df.to_csv("data/another_df.csv")
# df.head()